In [1]:
from os.path import join

def setup_file_system(in_colab):
    if in_colab:
        from google.colab import drive

        # Set the base and mount path
        MOUNT_PATH_DRIVE = '/content/drive'
        BASE_PATH = join(
            MOUNT_PATH_DRIVE, 
            "MyDrive/project_asr"
        )

        # Mount the google drive
        drive.mount(MOUNT_PATH_DRIVE)

        return BASE_PATH

    else:
        return "/workspaces/project_automated_sound_recognition"

In [2]:
import sys
from os import chdir
from os.path import join

# Method to check if the notebook is running in colab or local
IN_COLAB = 'google.colab' in sys.modules

# Set the base path of the project
BASE_PATH = setup_file_system(IN_COLAB)

# Set the base path of the project
chdir(join(BASE_PATH, "src/"))

In [3]:
%load_ext autoreload
%autoreload 2

# Imports
# Utils
import matplotlib as plt
import numpy as np
import wandb
import sys
import importlib
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import datetime
import json
from sklearn.metrics import accuracy_score, confusion_matrix


# DL libraries
import torch
import torch.optim as optim
from torch import nn
import torch.utils.data 
from torch.utils.data import DataLoader

# User libraries
from dataset.audio_sample_dataset import AudioSampleDataset
from model.baseline_model import BaselineModel
from trainer.trainer import train_classification_model
from validator.validator import validate_classification_model
from util import config, util_functions, model_management

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
augmentations = {
    'pitch_shift': {
        'enabled': False,
    },
    'noise': {
        'enabled': True,
        'p': 0.25,
        'min_amplitude': 0.001,
        'max_amplitude': 0.015,
    },
    'mixup': {
        'enabled': True,
        'p': 0.25,
        'alpha': 0.2,
    },
    'freq_mask': {
        'enabled': True,
        'p': 0.25,
        'freq_mask_param': 5,
    },
    'time_mask': {
        'enabled': True,
        'p': 0.25,
        'time_mask_param': 10,
    }
}

test_augmentations = {
    'pitch_shift': {
        'enabled': False,
    },
    'noise': {
        'enabled': False,
    },
    'mixup': {
        'enabled': False,
    },
    'freq_mask': {
        'enabled': False,
    },
    'time_mask': {
        'enabled': False,
    }
}

In [5]:
# Get the train and test data
train_dataset = AudioSampleDataset(
        join(BASE_PATH, config.TRAIN_DATA_PATH),
        augmentations
    )
test_dataset = AudioSampleDataset(
        join(BASE_PATH, config.TEST_DATA_PATH),
        test_augmentations
    )

# Place in dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1)

## Setup

In [6]:
# Clear gpu cache
torch.cuda.empty_cache()

# Get the model
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(in_features=512, out_features= 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(in_features=256, out_features=len(config.LABELS)),
    nn.Softmax(dim= 1)
)
model.to(config.DEVICE)

# Set the optimizer
optimizer = optim.Adam(model.parameters(), lr=config.LR)

# Set the loss fn
criteria = nn.CrossEntropyLoss()

# Set the gradient scaler
grad_scaler = torch.cuda.amp.grad_scaler.GradScaler()

# Setup weights and biasses
wandb.login()

# Get the current time for the checkpoint name
now = datetime.datetime.now()

# Set the wandb experiment name
experiment_name = util_functions.generate_run_name_from_config(augmentations)

# Start wandb
wandb.init(
    settings=wandb.Settings(start_method="fork"),
    project="project_asr", 
    name=experiment_name, 
    config={
        "learning_rate": config.LR,
        "batch_size": config.BATCH_SIZE,
        "epochs": config.EPOCHS,
        "augmentations": json.dumps(augmentations),
    }
)


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: robberdg. Use `wandb login --relogin` to force relogin


## Training

In [7]:
# Set the variables to keep track of the best model
best_validation_loss = 10000
best_model_state = model.state_dict()

for epoch in range(0,50):
  # Set the model in training mode
  model.train()
  
  # Train the model
  total_train_loss_this_epoch = train_classification_model(
      model,
      optimizer,
      criteria,
      grad_scaler,
      train_dataloader
  )
  
  # Set the model in evaluation mode
  model.eval()

  # Validate the model
  total_val_loss_this_epoch, pred_classes, true_classes = validate_classification_model(
      model,
      criteria,
      test_dataloader,
  )

  # Calculate the loss values
  train_loss_this_epoch = total_train_loss_this_epoch/len(train_dataloader.dataset)
  val_loss_this_epoch = total_val_loss_this_epoch/len(test_dataloader.dataset)

  # Calculate the accuracy
  acc_avg = accuracy_score(true_classes, pred_classes)

  # Calculate acc per class
  matrix = confusion_matrix(true_classes, pred_classes)
  acc_per_class = matrix.diagonal()/matrix.sum(axis=1)


  # Log the train loss this epoch
  wandb.log({
      'train_loss': train_loss_this_epoch,
      'val_loss': val_loss_this_epoch,
      'acc': acc_avg,
      'acc_airport': acc_per_class[0],
      'acc_shopping_mall': acc_per_class[1],
      'acc_metro_station': acc_per_class[3],
      'acc_street_pedestrian': acc_per_class[3],
      'acc_public_square': acc_per_class[4],
      'acc_street_traffic': acc_per_class[5],
      'acc_tram': acc_per_class[6],
      'acc_bus': acc_per_class[7],
      'acc_metro': acc_per_class[8],
      'acc_park': acc_per_class[9],
  })

  print(f'epoch: {epoch}, train_loss: {train_loss_this_epoch}, val_loss: {val_loss_this_epoch}, acc: {acc_avg}')

  # If this is the best performing model yet, save it
  if val_loss_this_epoch < best_validation_loss:
    # Update the score
    best_validation_loss = val_loss_this_epoch

    now = datetime.datetime.now()

    # Save the model
    checkpoint_path = join(
      BASE_PATH, 
      config.MODEL_CHECKPOINT_PATH, 
      f'{experiment_name}.pth'
    )
    best_model_state = model_management.save_model(model, checkpoint_path, False, '')

100%|██████████| 2000/2000 [00:27<00:00, 73.78it/s]


epoch: 0, train_loss: 0.018155316305160523, val_loss: 2.292484007358551, acc: 0.164


100%|██████████| 2000/2000 [00:27<00:00, 73.99it/s]


epoch: 1, train_loss: 0.01798256778717041, val_loss: 2.2670140978693962, acc: 0.199


100%|██████████| 2000/2000 [00:26<00:00, 74.40it/s]


...

epoch: 47, train_loss: 0.014194830524921417, val_loss: 2.069194474697113, acc: 0.3905


100%|██████████| 2000/2000 [00:26<00:00, 74.60it/s]


epoch: 48, train_loss: 0.014163315749168397, val_loss: 2.065612910568714, acc: 0.392


100%|██████████| 2000/2000 [00:26<00:00, 74.21it/s]

epoch: 49, train_loss: 0.014133450412750244, val_loss: 2.0686042371988296, acc: 0.3855


In [8]:
now = datetime.datetime.now()

# Save the final model
checkpoint_path = join(
    BASE_PATH, 
    config.MODEL_CHECKPOINT_PATH, 
    f'{experiment_name}.pth'
)
best_model_state = model_management.save_model(model, checkpoint_path, True, f'model_{experiment_name}')

In [9]:
# Mark the run as finished
wandb.finish()

acc,▁▂▃▅▅▅▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇███▇██████████████
acc_airport,▁▁▃▄▅▆▇▆▆▅▅▅▅▅▅▆▆▅▅▆▅▆▆▇▆▅▆▅▇▆█▆▅▆▆▇█▆▆▇
acc_bus,▁▁▄▇▅▅▄▃▃▂▃▃▃▃▃▄▄▅▄▅▆▆▆▄▅▆▆▆▇▆▇▆█▇▇▇█▇▇▆
acc_metro,▂▂▁▁▁▂▄▅████▇▇▇▇▇▇▇▇▆▇▆▇▆▆▇▆▆▆▇▇▆▇▇▇▆▆▇▇
acc_metro_station,▁▁▁▂▁▁▁▂▃▅▅▆▇▇▆▅▆▇▆▅▆█▇▇██▆▇▆█▇▇▇▇▆▆▅▇▇▇
acc_park,▄▇▆▆▄▁▃▄▃▄▅▃▅▄▆▅▅▆▅▇▇▅▆▅▄▅█▇█▇▆█▇▆█▇██▆▅
acc_public_square,▄▂▁▁▁▁▁▁▁▁▁▁▁▂▂▃▄▄▄▅▄▄▅▅▅▅▅▆▅▆▆▆▆▇▇▆▇▇██
acc_shopping_mall,▁▁▁▄██▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▆▆▆▆▆▅▅▅▆▆▆▆▆▅▆▅▅
acc_street_pedestrian,▁▁▁▂▁▁▁▂▃▅▅▆▇▇▆▅▆▇▆▅▆█▇▇██▆▇▆█▇▇▇▇▆▆▅▇▇▇
acc_street_traffic,▁▅██▇▆▆▆▇▇▇▆▇▆▇▇▆▆▆▆▇▆▆▆▆▆▆▆▆▆▆▆▆▆▅▆▆▆▆▆
acc_tram,▁▃▃▄▆▇██▆▆▆▆▆▇▆▆▆▅▅▅▅▄▅▅▅▄▄▅▄▄▄▄▄▄▄▃▄▄▄▄
